In [1]:
# 1. Install Library Streamlit dan Ngrok
!pip install -q streamlit pyngrok

# 2. Download Gambar Contoh (Agar siap diupload)
import os
import requests

def download_file(url, filename):
    if not os.path.exists(filename):
        print(f"Mendownload {filename}...")
        r = requests.get(url)
        with open(filename, 'wb') as f:
            f.write(r.content)
        print("Selesai.")

# Download Wolf (Content) & Starry Night (Style)
download_file('https://storage.googleapis.com/download.tensorflow.org/example_images/Golden_Gate_Bridge.jpg', 'contoh_content.jpg')
download_file('https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg', 'contoh_style.jpg')

print("\nSIAP! Gambar 'contoh_content.jpg' dan 'contoh_style.jpg' sudah ada di folder file (kiri).")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 27.1 MB/s eta 0:00:00
Mendownload contoh_content.jpg...
Selesai.
Mendownload contoh_style.jpg...
Selesai.

SIAP! Gambar 'contoh_content.jpg' dan 'contoh_style.jpg' sudah ada di folder file (kiri).


In [2]:
%%writefile nst_core.py
import tensorflow as tf
import numpy as np
from PIL import Image
import os

# --- KONFIGURASI ---
MAX_DIM = 512  # Ukuran max gambar (biar tidak terlalu berat di Colab)
STYLE_WEIGHT = 1e-2
CONTENT_WEIGHT = 1e4

# --- FUNGSI VGG (HARUS ADA DI SINI) ---
def vgg_layers(layer_names):
    """Memuat model VGG19 untuk ekstraksi fitur"""
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    outputs = [vgg.get_layer(name).output for name in layer_names]
    model = tf.keras.Model([vgg.input], outputs)
    return model

# --- FUNGSI PEMBANTU ---
def load_img(path_to_img):
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = MAX_DIM / long_dim
    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

def tensor_to_image(tensor):
    tensor = tensor * 255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor) > 3:
        tensor = tensor[0]
    return Image.fromarray(tensor)

def gram_matrix(input_tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result / num_locations

# --- MODEL UTAMA ---
class StyleContentModel(tf.keras.models.Model):
    def __init__(self, style_layers, content_layers):
        super(StyleContentModel, self).__init__()
        self.vgg = vgg_layers(style_layers + content_layers)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.num_style_layers = len(style_layers)
        self.vgg.trainable = False

    def call(self, inputs):
        inputs = inputs * 255.0
        preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
        outputs = self.vgg(preprocessed_input)
        style_outputs, content_outputs = (outputs[:self.num_style_layers],
                                          outputs[self.num_style_layers:])
        return style_outputs, content_outputs

# --- FUNGSI RUN (YANG DIPANGGIL APP) ---
def run_style_transfer(content_path, style_path, num_iterations=100):
    # Setup Layers
    content_layers = ['block5_conv2']
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

    # Load Gambar
    content_image = load_img(content_path)
    style_image = load_img(style_path)

    # Setup Model & Optimizer
    extractor = StyleContentModel(style_layers, content_layers)
    optimizer = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

    # Target
    style_targets = extractor(style_image)[0]
    style_targets = [gram_matrix(style_target) for style_target in style_targets]
    content_targets = extractor(content_image)[1]

    # Variable Image (Gambar yang akan diubah)
    image = tf.Variable(content_image)

    # --- FUNGSI TRAIN STEP (DALAM LINGKUP RUN AGAR AMAN) ---
    @tf.function
    def train_step(image):
        with tf.GradientTape() as tape:
            outputs = extractor(image)
            style_outputs = outputs[0]
            content_outputs = outputs[1]

            style_loss = tf.add_n([tf.reduce_mean((gram_matrix(so)-st)**2)
                                   for so, st in zip(style_outputs, style_targets)])
            style_loss *= STYLE_WEIGHT / len(style_outputs)

            content_loss = tf.add_n([tf.reduce_mean((co-ct)**2)
                                     for co, ct in zip(content_outputs, content_targets)])
            content_loss *= CONTENT_WEIGHT / len(content_outputs)

            loss = style_loss + content_loss

        grad = tape.gradient(loss, image)
        optimizer.apply_gradients([(grad, image)])
        image.assign(tf.clip_by_value(image, 0.0, 1.0))

    # Loop Training
    import time
    start = time.time()
    for i in range(num_iterations):
        train_step(image)
        # Optional: Print progress setiap 100 step (bisa dilihat di log Colab)
        if (i+1) % 50 == 0:
            print(f"Step {i+1}/{num_iterations} selesai...")

    return tensor_to_image(image)

Writing nst_core.py


In [3]:
%%writefile app.py
import streamlit as st
from PIL import Image
import os
from nst_core import run_style_transfer

st.set_page_config(page_title="NST Magic App", layout="wide")

st.title("🎨 AI Artist: Gabungkan Gaya Lukisan")
st.markdown("Upload foto biasa dan gambar lukisan, AI akan menggabungkannya!")

col1, col2 = st.columns(2)

uploaded_content = None
uploaded_style = None

with col1:
    uploaded_content = st.file_uploader("1. Upload Foto Utama (Content)", type=["jpg", "png", "jpeg"])
    if uploaded_content:
        st.image(uploaded_content, caption="Foto Utama", use_column_width=True)

with col2:
    uploaded_style = st.file_uploader("2. Upload Gaya Lukisan (Style)", type=["jpg", "png", "jpeg"])
    if uploaded_style:
        st.image(uploaded_style, caption="Gaya Lukisan", use_column_width=True)

st.markdown("---")

if st.button("✨ Mulai Sihir (Start Process)", type="primary"):
    if uploaded_content and uploaded_style:
        # Simpan sementara
        with open("temp_content.jpg", "wb") as f:
            f.write(uploaded_content.getbuffer())
        with open("temp_style.jpg", "wb") as f:
            f.write(uploaded_style.getbuffer())

        with st.spinner('Sedang melukis... (Butuh waktu sekitar 1-2 menit di Colab)'):
            try:
                # Jalankan Style Transfer (100 iterasi biar cepat buat demo)
                result_image = run_style_transfer("temp_content.jpg", "temp_style.jpg", num_iterations=100)

                st.success("Selesai!")
                st.subheader("Hasil Karya:")
                st.image(result_image, use_column_width=True)

                # Tombol Download
                result_image.save("hasil_art.jpg")
                with open("hasil_art.jpg", "rb") as file:
                    st.download_button(
                        label="Download Hasil",
                        data=file,
                        file_name="hasil_art.jpg",
                        mime="image/jpeg"
                    )
            except Exception as e:
                st.error(f"Terjadi error: {e}")
    else:
        st.warning("Mohon upload kedua gambar di atas dulu ya!")

Writing app.py


In [8]:
from pyngrok import ngrok
import time

# 1. Matikan koneksi ngrok sebelumnya (biar tidak error port in use)
ngrok.kill()

# 2. Masukkan Token Anda
NGROK_AUTH_TOKEN = "377A3zSIr8IdJtgCboRfIFgwT0f_3CnwjWVV8uD5evxQTRfwj"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# 3. Jalankan Streamlit di Background
get_ipython().system_raw('streamlit run app.py --server.port 8501 &')

# Tunggu sebentar biar server nyala
time.sleep(3)

# 4. Buka Tunnel ke Publik
try:
    public_url = ngrok.connect(8501).public_url
    print("="*50)
    print(f"KLIK LINK INI UNTUK BUKA APLIKASI: {public_url}")
    print("="*50)
except Exception as e:
    print(f"Error Ngrok: {e}")

KLIK LINK INI UNTUK BUKA APLIKASI: https://nonindependent-reyna-unsubmergible.ngrok-free.dev
